In [1]:
!pip install langchain_community
!pip install faiss-gpu
!pip install rank_bm25 
!pip install ragas
!pip install datasets
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.1

In [2]:
import os
os.getcwd()

'/kaggle/working'

In [4]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('/kaggle/input/star-wars/Star Wars - Brotherhood Mike Chen.pdf') as pdf:
    # Open the text file for writing
    with open('knowledge_base.txt', 'w', encoding='utf-8') as output:
        # Iterate over pages 10 to 349 (0-indexed, so subtract 1)
        for i in range(9, 349):  # Page 10 is index 9
            page = pdf.pages[i]
            text = page.extract_text() or ""  # Handle cases where text is None
            output.write(text + '\n')  # Write text to file with a newline


In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
from rank_bm25 import BM25Okapi
import numpy as np
import warnings

from datasets import Dataset
warnings.filterwarnings("ignore")
import textwrap

In [6]:
# 1. Load your text files
file_paths = ["/kaggle/working/knowledge_base.txt"]
documents = []

for file_path in file_paths:
    loader = TextLoader(file_path)
    docs = loader.load()
    documents.extend(docs)

# 2. Define chunking parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,        # You can try 100, 250, 512, etc.
    chunk_overlap=0       # Try 0, 50, 100, etc.
)

# 3. Split the documents
chunks = text_splitter.split_documents(documents)

# 4. Output result
print(f"Total chunks: {len(chunks)}")
print(f"First chunk content:\n{chunks[0].page_content}")

# Optional: Save the chunks to a file
with open("chunked_output.txt", "w", encoding='utf-8') as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i + 1} ---\n")
        f.write(chunk.page_content + "\n\n")


Total chunks: 2671
First chunk content:
A long time ago in a galaxy far, far away….
The CLONE WARS have erupted. Caught off guard by the quickly
expanding conflict, the overwhelmed Jedi Order has rushed the
advancement of Padawans to better integrate into the Grand Army of the


In [7]:
# Prepare documents and their metadata
texts = [chunk.page_content for chunk in chunks]
metadata = [chunk.metadata for chunk in chunks]
print(len(texts))

2671


In [8]:
# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Create FAISS vector database
vectordb = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index to disk for later use
vectordb.save_local("faiss_index")

# Check the number of stored documents
print(f"Number of documents in the vector store: {vectordb.index.ntotal}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of documents in the vector store: 2671


In [9]:
# BM25 Indexing
tokenized_texts = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_texts)

def reciprocal_rank_fusion(results_bm25, results_embedding, k=2):
    scores = {}

    # Use document content or metadata as the key
    for rank, (doc, score) in enumerate(results_bm25):
        doc_id = doc.page_content  # Or use doc.metadata.get("source", "unknown") if available
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("BM25", scores[doc_id])

    for rank, (doc, score) in enumerate(results_embedding):
        doc_id = doc.page_content  # Use the same identifier
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank+1) # (k + rank + 1)
        print("Dense", scores[doc_id])

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)


# Extract page content and metadata properly
def format_response(doc):
    return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

In [10]:
# Retrieve function
def retrieve(query, k=3):
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)
    results_embedding = sorted(results_embedding, key=lambda x: x[1], reverse=True)
    
    print("============Dense Embeddings=============")
    for doc, score in results_embedding:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")

    # Get BM25 scores for all documents and sort to get top-k results
    results_bm25 = [(idx, bm25.get_scores(query.split())[idx]) for idx in range(len(texts))]
    results_bm25 = sorted(results_bm25, key=lambda x: x[1], reverse=True)[:k]  # Keep only top-k results
    # Convert BM25 results to (Document, score) format
    results_bm25_docs = [(Document(page_content=texts[idx], metadata=metadata[idx]), score) for idx, score in results_bm25]
   
    print("************BM25 Results*************")
    for doc, score in results_bm25_docs:
        print(f"page {doc.metadata.get('page','Unknown')} - Score: {score:.4f} - {doc.page_content[:100]}...")
    
    # Create a lookup dictionary {document content -> Document object}
    doc_lookup = {doc.page_content: doc for doc, _ in results_bm25_docs}
    doc_lookup.update({doc.page_content: doc for doc, _ in results_embedding})

    # Fuse results
    fused_results = reciprocal_rank_fusion(results_bm25_docs, results_embedding)
    
    # Format results, ensuring document IDs are mapped back to actual Documents
    return [format_response(doc_lookup[doc_id]) for doc_id, _ in fused_results if doc_id in doc_lookup]

    #fused_results = reciprocal_rank_fusion(results_bm25, results_embedding)
    #return [(texts[idx], metadata[idx]["page"] if "page" in metadata[idx] else "Unknown") for idx, _ in fused_results]


In [17]:
# Query example
question = "Who carry out the bombing on Cato Neimoidia in Brotherhood?"
retrieved_responses = retrieve(question, k=10)

============Dense Embeddings=============
page Unknown - Score: 0.2996 - “Perhaps.” Dex stretched out, upper arms reaching overhead while his
lower arms clutched his belly. ...
page Unknown - Score: 0.2996 - the best interests of Neimoidians.”
“You have sold out your people.”
“This bomb would have murdered ...
page Unknown - Score: 0.2995 - Murder and destruction. That was her life for so many years, doing the
will of her government to car...
page Unknown - Score: 0.2982 - Separatists. Just as the datapad showed materials and protocols matching
Republic bombs used by the ...
page Unknown - Score: 0.2913 - conspiring against Cato Neimoidia. Of covering up evidence. Of breaking
his promises. And of a much ...
page Unknown - Score: 0.2910 - both sides of the case. But one thing that must be considered: What were
the conditions that caused ...
page Unknown - Score: 0.2908 - superiority in front of other Neimoidian guards. “Would you prefer me to
rain blaster bolts on every...
page Unknown 

In [12]:
# # Query processing
# question = "What was the cause of the bombing on Cato Neimoidia in Brotherhood?"
# retriever = vectordb.as_retriever(search_kwargs={"k": 10})
# docs = retriever.get_relevant_documents(question)

# # Print results
# for i, doc in enumerate(docs, 1):
#     page_number = doc.metadata.get('page', 'Unknown')
#     # print(f"Document {i} - Page {page_number} - Score: {doc.metadata.get('score', 'N/A')}")
#     print(doc.page_content[:])  # Print first 500 characters of each result
#     print("-" * 80)

In [18]:
model_name = "tiiuae/Falcon3-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #device_map='cuda'
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# Create a pipeline
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
return_full_text=False,
max_new_tokens=5000,
do_sample=False
)

Device set to use cuda:0


In [15]:
# # Extract page content and metadata properly
# def format_response(doc):
#     return f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}"

# # Handle cases where fewer than 3 results are returned
# retrieved_responses = [format_response(doc) for doc in docs[:3]]
# while len(retrieved_responses) < 3:
#     retrieved_responses.append("No relevant information.")  # Fill missing slots

In [20]:
# Construct the RAG prompt
prompt = f"""
You are an AI assistant tasked with answering questions based on retrieved knowledge.

### **Retrieved Information**:
1. {retrieved_responses[0]}
2. {retrieved_responses[1]}
3. {retrieved_responses[2]}
4. {retrieved_responses[3]}
5. {retrieved_responses[4]}
6. {retrieved_responses[5]}
7. {retrieved_responses[6]}
8. {retrieved_responses[7]}
9. {retrieved_responses[8]}
10. {retrieved_responses[9]}

### **Question**:
{question}

### **Instructions**:
- Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
- If the responses are **contradictory**, mention the different perspectives.
- If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
"""



In [21]:
for i in range(0,len(retrieved_responses)):
    print(retrieved_responses[i])
    print("-------")

Page Unknown: different steps forward. So many variables came into play—who stole the
equipment? Who had they shown it to? Was it contained to Cato Neimoidia
or had it gotten bigger, dragging both the CIS and Republic into this?
-------
Page Unknown: “Perhaps.” Dex stretched out, upper arms reaching overhead while his
lower arms clutched his belly. “It’s simple, really. Cato Neimoidia is the
base of operations for the Trade Federation. Long memories, those
Neimoidians.”
-------
Page Unknown: Cato Neimoidia as the conflict unfolded?” Arms raised, he gestured
outward, upward, each movement creating another wave of noise from the
crowd. “Would Cato Neimoidia even still be a target? As we determine the
-------
Page Unknown: the best interests of Neimoidians.”
“You have sold out your people.”
“This bomb would have murdered civilians!”
“Keep telling yourself that. Take the easy way out.” Ketar spat at Ruug’s
-------
Page Unknown: bombing from the library terminal. “What can you tell me about

In [22]:
# Use Qwen2.5 3B with the correct message format
messages = [
    {"role": "user", "content": prompt}
]

# Generate output using the model
output = generator(messages)

# Print formatted response
print(textwrap.fill(output[0]["generated_text"], width=80))

The bombing at Cato Neimoidia in "Brotherhood" is a complex event with multiple
layers of intrigue and betrayal. According to Dex, who is a Neimoidian and
presumably involved in the Trade Federation, the base of operations for the
Federation, it appears that the bombing was orchestrated by individuals with
deep connections to the Neimoidians. He suggests that the bombing was a
calculated move to undermine the Neimoidians' control over their operations.
However, other characters and perspectives offer different insights. For
instance, the character who was supposed to pay Ketar before revealing
information about the bombing implies a level of betrayal and suggests that the
bombing was not just a random act but part of a larger scheme. This character's
statement about selling out their people and taking the easy way out indicates a
strategic decision to dismantle the Neimoidians' influence.  Furthermore, the
character who was attacked by Separatists and the one who was supposed to go to
